In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
from math import * 
import torch

print("The current directory is: ")
print(os.getcwd())
if not os.getcwd().endswith("Abaqus-Hardening-Seq-2-Seq-Project"):
    # Move up two directories
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
print("The current directory is: ")
print(os.getcwd())

The current directory is: 
c:\Users\springnuance\Desktop\Abaqus-Hardening-Seq-2-Seq-Project\notebooks\CP1000_RD_20C
The current directory is: 
c:\Users\springnuance\Desktop\Abaqus-Hardening-Seq-2-Seq-Project


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from configs.chosen_project import *
from src.stage1_global_configs import *

chosen_project_path = "configs/global_config_CP1000_RD_20C.json"

global_configs = main_global_configs(chosen_project_path)

all_paths = global_configs['all_paths']
objectives = global_configs['objectives']


= Stage 1: Loading configs and all paths =

Welcome to Abaqus Seq2Seq flow curve calibration project

The configurations you have chosen: 

+--------------------------+------------------------------------------------------------------+
|      Global Configs      |                           User choice                            |
+--------------------------+------------------------------------------------------------------+
|         PROJECT          |                          CP1000_RD_20C                           |
|        OBJECTIVES        |        CHD2, CHD4, NDBR2p5, NDBR6, NDBR15, NDBR40, SH115         |
|       PROJECT_PATH       | c:\Users\springnuance\Desktop\Abaqus-Hardening-Seq-2-Seq-Project |
|    TRAINING_DATA_PATH    |                   training_data/CP1000_RD_20C                    |
|         LOG_PATH         |                      log/CP1000_RD_20C.txt                       |
|       MODELS_PATH        |                       models/CP1000_RD_20C                    

##### In this stage, we construct the torch tensors from the FD curves, the flow curves for both simulation and the experimental FD curves. 

source_sequences is of shape (num_samples, source_len, num_objectives). It is the combination of FD curve. Later, num_samples would be the batch_size

target_sequences is of shape (num_samples, target_len, 1). All FD curves share one single flow curve. Later num_samples would be the batch_size 

In [6]:
results_init_common_path = all_paths["results_init_common_path"]
training_data_path = all_paths["training_data_path"]
targets_path = all_paths["targets_path"]

model_config = global_configs["model_config"]

interpolated_displacement_len = global_configs["interpolated_displacement_len"]

source_sequence = []

for i, objective in enumerate(objectives):
    source_sequence_one_sim = []
    if os.path.exists(f"{results_init_common_path}/{objective}/FD_curves_interpolated.npy"):
        # Plotting the mean simulated FD curve
        FD_curves_interpolated = np.load(f"{results_init_common_path}/{objective}/FD_curves_interpolated.npy", allow_pickle=True).tolist()
        for params_tuple, FD_curve_interpolated in FD_curves_interpolated.items():
            sim_force_interpolated = FD_curve_interpolated['force']
            source_sequence_one_sim.append(sim_force_interpolated)
        source_sequence.append(source_sequence_one_sim)

source_sequence = np.array(source_sequence)
# Convert to tensor
source_sequence = torch.tensor(source_sequence)
source_sequence = source_sequence.permute(1, 2, 0)
print("The source sequence is constructed with shape (num_sims, source_len, num_objectives):")
print(source_sequence.shape)
            
# Now we need to verify if this source sequence is constructed correctly

for i, objective in enumerate(objectives):
    
    if os.path.exists(f"{results_init_common_path}/{objective}/FD_curves_interpolated.npy"):
        # Plotting the mean simulated FD curve
        FD_curves_interpolated = np.load(f"{results_init_common_path}/{objective}/FD_curves_interpolated.npy", allow_pickle=True).tolist()
        for sim_index, (params_tuple, FD_curve_interpolated) in enumerate(FD_curves_interpolated.items()):
            sim_force_interpolated = FD_curve_interpolated['force']
            sim_force_interpolated = torch.tensor(sim_force_interpolated)
            assert torch.allclose(source_sequence[sim_index, :, i], sim_force_interpolated)
    
# Now we save the source sequence
torch.save(source_sequence, f"{training_data_path}/initial_source_sequence.pt")

The source sequence is constructed with shape (num_sims, source_len, num_objectives):
torch.Size([256, 100, 7])


In [7]:
### Now we construct the target sequence

target_sequence = []

if os.path.exists(f"{results_init_common_path}/initial_sampled_true_stress.npy"):

    initial_sampled_true_stress = np.load(f"{results_init_common_path}/initial_sampled_true_stress.npy", allow_pickle=True).tolist()
    target_sequence = torch.tensor(initial_sampled_true_stress)

# turn it into (num_sims, target_len, 1)
target_sequence = target_sequence.unsqueeze(-1)
print("The target sequence is constructed with shape (num_sims, target_len, 1):")
print(target_sequence.shape)

# Now we save the target sequence
# target length totally depends on true_plastic_strain_config, we should not interpolate them

torch.save(target_sequence, f"{training_data_path}/initial_target_sequence.pt")

The target sequence is constructed with shape (num_sims, target_len, 1):
torch.Size([256, 100, 1])


In [8]:
# Now we construct the exp_source_sequence, which is used the prediction of flow curve
# There is only 1 experimental data so it should have shape (1, source_len, num_objectives)

targets_path = all_paths["targets_path"]
exp_source_sequence = []

for i, objective in enumerate(objectives):
    FD_curve_final_interpolated = pd.read_excel(f"{targets_path}/{objective}/FD_curve_final_interpolated.xlsx", engine='openpyxl')
    exp_force_interpolated = FD_curve_final_interpolated['force/N'].values
    exp_source_sequence.append(exp_force_interpolated)

# Convert to tensor
exp_source_sequence = torch.tensor(exp_source_sequence).permute(1,0)
exp_source_sequence = exp_source_sequence.unsqueeze(0)
print("The exp_source_sequence is constructed with shape (1, source_len, num_objectives):")
print(exp_source_sequence.shape)

# Verify if this exp_source_sequence is constructed correctly
for i, objective in enumerate(objectives):
    FD_curve_final_interpolated = pd.read_excel(f"{targets_path}/{objective}/FD_curve_final_interpolated.xlsx", engine='openpyxl')
    exp_force_interpolated = FD_curve_final_interpolated['force/N'].values
    exp_force_interpolated = torch.tensor(exp_force_interpolated)
    assert torch.allclose(exp_source_sequence[0, :, i], exp_force_interpolated)
    
# Now we save the exp_source_sequence
torch.save(exp_source_sequence, f"{training_data_path}/exp_source_sequence.pt")

The exp_source_sequence is constructed with shape (1, source_len, num_objectives):
torch.Size([1, 100, 7])


In [9]:
initial_test_ratio = model_config["initial_test_ratio"]

# Now we split the source_sequence, target_sequence and exp_source_sequence into training and testing
# There is no randomization
num_sims = source_sequence.shape[0]

num_test_sims = ceil(num_sims * initial_test_ratio)
num_train_sims = num_sims - num_test_sims

train_source_sequence = source_sequence[:num_train_sims]
train_target_sequence = target_sequence[:num_train_sims]
test_source_sequence = source_sequence[num_train_sims:]
test_target_sequence = target_sequence[num_train_sims:]

print("The training and testing source and target sequences are constructed with shapes:")
print("train_source_sequence shape:", train_source_sequence.shape)
print("train_target_sequence shape:", train_target_sequence.shape)
print("test_source_sequence shape:", test_source_sequence.shape)
print("test_target_sequence shape:", test_target_sequence.shape)

torch.save(train_source_sequence, f"{training_data_path}/initial_train_source_sequence.pt")
torch.save(train_target_sequence, f"{training_data_path}/initial_train_target_sequence.pt")
torch.save(test_source_sequence, f"{training_data_path}/initial_test_source_sequence.pt")
torch.save(test_target_sequence, f"{training_data_path}/initial_test_target_sequence.pt")

The training and testing source and target sequences are constructed with shapes:
train_source_sequence shape: torch.Size([192, 100, 7])
train_target_sequence shape: torch.Size([192, 100, 1])
test_source_sequence shape: torch.Size([64, 100, 7])
test_target_sequence shape: torch.Size([64, 100, 1])


##### However, due to the nature of the flow curve, which is always monotonically increasing, using the flow curve as the target sequence would not be a good idea. Instead, we use the derivative of the flow curve as the target sequence. To be compatible, the FD curves are also differentiated. 

The differentiation is simply done by subtracting the current value from the next value, reducing the length of the sequence by 1. When training the Seq2Seq models, we can use a ReLU activation function to ensure the target sequence is always positive, reflecting the positive incremental flow curve

Question: how can we reconstruct the curves when we do not know the first value?

We would train two separate models, one to learn the incremental change, and one to solely learn the first N values. This N values is "divided_index" from model_config

In [10]:
divided_index = model_config["divided_index"]
print("The divided index is:", divided_index)

train_source_sequence_diff = train_source_sequence[:, divided_index+1:, :] - train_source_sequence[:, divided_index:-1, :]
train_target_sequence_diff = train_target_sequence[:, divided_index+1:, :] - train_target_sequence[:, divided_index:-1, :]
test_source_sequence_diff = test_source_sequence[:, divided_index+1:, :] - test_source_sequence[:, divided_index:-1, :]
test_target_sequence_diff = test_target_sequence[:, divided_index+1:, :] - test_target_sequence[:, divided_index:-1, :]

print("The training and testing source and target sequences are constructed with shapes:")
print("train_source_sequence_diff shape:", train_source_sequence_diff.shape)
print("train_target_sequence_diff shape:", train_target_sequence_diff.shape)
print("test_source_sequence_diff shape:", test_source_sequence_diff.shape)
print("test_target_sequence_diff shape:", test_target_sequence_diff.shape)

torch.save(train_source_sequence_diff, f"{training_data_path}/initial_train_source_sequence_diff.pt")
torch.save(train_target_sequence_diff, f"{training_data_path}/initial_train_target_sequence_diff.pt")
torch.save(test_source_sequence_diff, f"{training_data_path}/initial_test_source_sequence_diff.pt")
torch.save(test_target_sequence_diff, f"{training_data_path}/initial_test_target_sequence_diff.pt")

train_source_sequence_first = train_source_sequence[:, :divided_index+1, :]
train_target_sequence_first = train_target_sequence[:, :divided_index+1, :]
test_source_sequence_first = test_source_sequence[:, :divided_index+1, :]
test_target_sequence_first = test_target_sequence[:, :divided_index+1, :]

print("The training and testing source and target sequences are constructed with shapes:")
print("train_source_sequence_first shape:", train_source_sequence_first.shape)
print("train_target_sequence_first shape:", train_target_sequence_first.shape)
print("test_source_sequence_first shape:", test_source_sequence_first.shape)
print("test_target_sequence_first shape:", test_target_sequence_first.shape)

torch.save(train_source_sequence_first, f"{training_data_path}/initial_train_source_sequence_first.pt")
torch.save(train_target_sequence_first, f"{training_data_path}/initial_train_target_sequence_first.pt")
torch.save(test_source_sequence_first, f"{training_data_path}/initial_test_source_sequence_first.pt")
torch.save(test_target_sequence_first, f"{training_data_path}/initial_test_target_sequence_first.pt")


The divided index is: 0
The training and testing source and target sequences are constructed with shapes:
train_source_sequence_diff shape: torch.Size([192, 99, 7])
train_target_sequence_diff shape: torch.Size([192, 99, 1])
test_source_sequence_diff shape: torch.Size([64, 99, 7])
test_target_sequence_diff shape: torch.Size([64, 99, 1])
The training and testing source and target sequences are constructed with shapes:
train_source_sequence_first shape: torch.Size([192, 1, 7])
train_target_sequence_first shape: torch.Size([192, 1, 1])
test_source_sequence_first shape: torch.Size([64, 1, 7])
test_target_sequence_first shape: torch.Size([64, 1, 1])
